# Measurements & Quantification Notebook

In [ ]:
# /// script
# requires-python = ">=3.10"
# dependencies = [
#     "matplotlib",
#     "numpy",
#     "scikit-image",
#     "scipy",
#     "tifffile",
#     "imagecodecs",
#     "pandas",
# ]
# ///

## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Overview</mark>

In this notebook, we will explore one possible approach to **extracting**, **measuring**, and **quantifying** information from **fluorescence microscopy images**, or, in other words, how to **go from images, to numbers, to plots**.

While fluorescence images can contain a wide range of information, in this notebook, as an example, we will focus on extracting **quantitative information** about the **intensity** of the fluorescence signal and the **morphology** of the objects in the image, mainly using the [**scikit-image**](https://scikit-image.org/docs/stable/) library.

In addition, we will learn how to save the extracted information as a **.csv file** using the [**pandas**](https://pandas.pydata.org/docs/) library.

The images we will use for this section can be downloaded from the <a href="" download> <i class="fas fa-download"></i> Classic Segmentation Dataset</a>. We will also need the **labelled images** that we created in the [Classic Segmentation](../05_segmentation/classic/classic.md) section.

<p class="alert alert-info">
    <strong>Note:</strong> The images from <strong>Classic Segmentation Dataset</strong> have been pre-processed to remove the background and correct for illumination variations (flat-field correction) and are therefore of type <strong>float32</strong>.
</p>

**TODO: UPDATE THE LINK TO THE DATASET AND TO THE CLASSIC SEGMENTATION**
**THE DATASET IS TOO BIG TO BE STORED ON GITHUB**

## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Importing libraries</mark>

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage
import tifffile

## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Load and Display Images</mark>

Let's load and display with `matplotlib` our raw and segmented images (labeled images):

## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Accessing single objects in the labeled image</mark>

Remember that a **labeled image** is an image where each pixel is assigned a **unique value** (or label) that corresponds to a specific object or region in the image, in our case, nuclei. This means that each nucleus in the image is represented by a unique integer value, and all pixels that belong to the same nucleus share the same value.

For example, let's try to create a new image that contains only the pixels of the first nucleus in the labeled image:

This new image we obtained is a **boolean binary mask** with value `False` for background and `True` for the nucleus number 1.

To verify that this is indeed a binary mask, we can query for the minimum and maximum value using the `np.min()` and `np.max()` functions:


## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Manual Extraction of Properties</mark>

Now we can for example use this mask to extract and print the intensity values of the pixels that belong to this nucleus from the original image.

We can now calculate and print for example the **mean intensity** using the `np.mean()` function:

Or we can measure and print the area in pixels of the nucleus 1, which simply corresponds to the total number of pixels that have the value `True` in the binary mask. We can do this by using the `np.sum()` function:

To convert the area to a more meaningful unit, such as micrometers squared, we need to know the pixel size of the image.

If we assume a pixel size of 0.325 micrometers, we can convert the area from pixels to square micrometers by multiplying by the square of the pixel size:

## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Automated Extraction of Properties</mark>

We could definitely implement a function that would automate the extraction of intensity values or the area for all nuclei in the labeled image, as well as implement more complex measurements but luckily, we don't have to do it ourselves, as there are libraries that can help us with this task and more in a very efficient way.

The [`skimage.measure`](https://scikit-image.org/docs/stable/api/skimage.measure.html) module from the **scikit-image** package provides several tools to measure properties of image regions. It is especially powerful when working with **labeled segmentation masks**, where each object in an image is assigned a unique integer label.

This module is a foundational tool for extracting morphological and intensity-based features from binary or labeled images.

A useful function in `skimage.measure` is [`regionprops`](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops):

**Parameters:**
- `label_image` — A 2D or 3D array where **each unique non-zero integer** corresponds to a labeled object (typically from `skimage.measure.label`).
- `intensity_image` *(optional)* — The raw grayscale image associated with the label mask, used to compute intensity features like mean or standard deviation.

For example:

```python
skimage.measure.regionprops(label_image, image)
```

**Returns:**
- A `list` of `RegionProperties` objects — one for each labeled object. **Each object contains attributes describing shape, position, and optionally intensity** and can be accessed with dot notation (e.g., `RegionProperties.area` or `RegionProperties.mean_intensity`).

In the table below, we summarize some of the common features that can be extracted using this function:

| Measurement | Description |
|-------------|-------------|
| area | Measures the number of pixels in the object (size) |
| perimeter | Approximates the length around the object |
| eccentricity | Quantifies elongation—how stretched the shape is |
| solidity | Indicates shape irregularity by comparing area to convex area |
| extent | Compares object area to its bounding box area |
| orientation | Measures the angle of the major axis of the object |
| mean_intensity | Averages pixel values inside the region (if intensity image is given) |
| max_intensity | Finds the brightest pixel value inside the object |
| intensity_image | Extracts the raw intensity image cropped to the object |
| centroid | Finds the object's center of mass (position in the image) |
| bbox | Gives the bounding box coordinates of the object |

Let's create a variable named `props` to store the result of the `skimage.measure.regionprops` function using the `labeled_mask` and the original `image` as inputs:

Since the `props` variable is a `list` of `RegionProperties`, we can access the properties of each object using indexing. For example, to access the properties of the first object (or the first nucleus), we can use `props[0]`.

From the [`regionprops`](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops) documentation (and as mentioned above), we can see that we can access a lot of properties for each `RegionProperties` in the list. For example, to get the **mean intensity** of the first region (nucleus), we can use `props[0].mean_intensity`:

As you can see, is the same value we obtained manually before.

<p class="alert alert-info">
    <strong>Note:</strong> To see all the available properties for <em>RegionProperties</em> use the <em>dir()</em> function which will list all the attributes and methods of the object: <em>dir(props[0])</em>. Ignore all the methods that start with <em>__</em> (double underscore) as they are special methods in Python.
</p>

Let's now extract the **mean intensity** of all nuclei and plot it using `matplotlib` in an histogram:

If you remember from the [Classic Segmentation](../05_segmentation/classic/classic.md) section, the dataset we are using contains images of **HeLa cells** stained with **DAPI**, a fluorescent dye that binds to DNA.

We know that:
- In normal condition, mammalian cells in the **G1** phase of the cell cycle are **diploid** and thus have two sets of chromosomes (**2n**).
- Cells in the **G2** phase and are **tetraploid** (**4n**).
- During **S** phase (DNA doubling), cells have a DNA content ranging between **2n** and **4n**.

Therefore, by looking at the DNA content (2n vs 4n) in a cell nucleus, it should be possible to discriminate between the cell cycle phases.

Since DAPI labels the DNA depending on the DNA content, by **measuring** the **total DAPI nuclear intensity (integrated density)**, we might be able to **plot the cell cycle phase** distribution of the entire population of cells.

The total intensity, also known as **integrated density**, is defined as the **sum of the pixel intensities within a region of interest**, which in our case is the nucleus. To calculate it we can use the `image_intensity` attribute of the `RegionProperties` object, which contains the values of all the pixels in the region of interest (the nucleus) and sum them up using the `np.sum()` function.

For example:

### <mark style="color: black; background-color: rgb(252,185,118); padding: 3px; border-radius: 5px;">✍️ Exercise: Plot DAPI Nuclear Integrated Density Distribution from the Entire Dataset</mark>

In this exercise, let's try to:
- create a `dataset_integrated_density_list` list variable
- write a `for loop` that will iterate over (load) all pairs of images (raw and labeled) in the dataset
    - extract the total integrated density of each nucleus in each image
    - store the results in a list named `dataset_integrated_density_list`
- plot the distribution of the integrated density values using `matplotlib` and a histogram (bin size of 250)

## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">From Properties to Tables</mark>

There is another useful function in the `skimage.measure` module called [`regionprops_table`](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops_table) that allows us to create a table of properties for each region in the labeled image.

Looking at the [documentation](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops_table), we can see that this function, instead of a `list`, returns a `dict` object (dictionary type as we've seen in [Data Structures: Dictionaries](../03_python_basics/python_basics_notebook.ipynb)) where each property is a `key` and the `values` are arrays containing the property values for each region in the labeled image.

The main attribute of this function are:
- **label_image**: The labeled image where each unique integer corresponds to a labeled object.
- **intensity_image** *(optional)*: The raw grayscale image associated with the label mask (used to compute intensity features).
- **properties** *(optional)*: A list of properties to include in the table. If not specified, all properties will be included.

For example:

```python
props_dict = regionprops_table(label_image, raw_image, properties=['area', 'perimeter'])
```

Let's see how we can use this in practice and create a DataFrame with the `image_intensity` and `area` properties for each nucleus in the labeled image.

First we need to reload one raw image and the corresponding labeled image:x

Create the dictionary with the `regionprops_table` function that includes the `label`, `image_intensity`, and `area` properties, and print the keys (remember this is a `dict`) of the results:

Since `props_dict` is a dictionary, we can access the properties of each region using the keys of the dictionary. Each key corresponds to a property, and the associated value is an array containing the property values for each region.

For example, to access the area of each region, we can do:

```python
areas = props_dict['area']
```

Similarly, we can access and print `image_intensity` property (print only the first three values):

There is not yet an evident advantage of using `regionprops_table` over `regionprops` at this point, but this method becomes useful when we want to create a table with all the properties of each region in the labeled image as well as save it as a `.csv` file. In fact, this `props_dict` output can be used to directly create a [`**pandas**`](https://pandas.pydata.org/docs/) DataFrame.

[**pandas**](https://pandas.pydata.org/docs/) is a powerful Python library for data manipulation and analysis. It provides high-performance, easy-to-use data structures and tools for working with structured data. The main data structure in pandas is the **DataFrame**, which is a 2-dimensional labeled data structure with columns of potentially different types (like numbers, strings, arrays, etc.).

Let's now create a DataFrame with the `props_dict` variable and print it:


We can notice that now we converted the `dict` file in a more tabular format, which is easier to work with for data analysis and manipulation tasks. The rows of the DataFrame correspond to the regions (nuclei) in the labeled image, and the columns correspond to the properties of each region.

For example we can now access the `image_intensity` property by the column name as key:

And if we want to get the integrated density values for the fist nucleus, we can do (similarly as before):

And, similarly to before, to get the integrated density values for all nuclei, we can use create a list where we sum the pixel intensities for each nucleus:

## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Manipulating tables as dataframes</mark>


Now that we have converted the measurements table into a DataFrame, we can start manipulating it.

For example, we can create a new column in the DataFrame with the integrated density per nucleus, and then display the first few rows of the DataFrame using the `head()` method:

Dataframe makes conditional filtering and selection easy. We can use the `loc[...]` method to select rows and columns <u>by their labels</u>.

For example, let's identify the percentage of nuclei that are larger than average and have high integrated density:

<div class="alert alert-info">
  <strong>Note:</strong> Using <em>.loc[...]</em> is the recommended and idiomatic way to filter rows in a pandas DataFrame. Here's why:
<ul>
  <li><strong>Explicit row filtering</strong>: Clearly signals that you're selecting rows based on a condition.</li>
  <li><strong>Preserves structure</strong>: Returns a full DataFrame with all metadata (index, column names), unlike <em>df[condition]</em> which may behave ambiguously.</li>
  <li><strong>Avoids chained indexing issues</strong>: Prevents bugs related to operating on a copy vs. a view.</li>
  <li><strong>Safe for assignment</strong>: Enables safe in-place updates (e.g., <em>df.loc[condition, 'col'] = value</em>).</li>
  <li><strong>Readable and standard</strong>: Immediately recognizable and understandable to other pandas users.</li>
</ul>
</div>


We can then extract the labels of large and bright nuclei from the DataFrame and create a binary mask showing only those selected nuclei in the original image using the `np.isin(against, for)` function.

Once the nuclei of interest have been selected in the dataframe, it is possible to visualize them in the labeled image.

It is also possible to select <u>by position</u> using `.iloc[...]`.

For example, to select the first 10 rows of the DataFrame, we can do:

To select the first 10 rows and the first 3 columns, we can do:

## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Saving dataframes to CSV</mark>

One of the most useful features of pandas is the ability to easily save data to various file formats. Let's save our measurements to a CSV file that can be opened in spreadsheet software like Excel:


The other way around, CSV files can then be loaded back into a pandas DataFrame using the `read_csv` method: